# Regularyzacja w modelu regresji - porównanie regresji grzbietowej i regresji Lasso

In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 10

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

**Regularyzacja grzbietowa** i **Lasso** są technikami, które są wykorzystywane do budowania **oszczędnych modeli**, w rozumieniu obecności zbyt dużej liczby predyktorów. 
Przez dużą liczbę predyktorów rozumiemy:

- *duża liczba predyktorów* to taka, która prowadzi do **przeuczenia modelu** (ang. *overfitting*) -- nawet tak niewielka liczba jak 10 zmiennych może prowadzić do przeuczenia,
    
- *duża liczba predyktorów* to taka, która może prowadzić do problemów z **wydajnością obliczeniową** -- przy obecnych możliwościach komputerów, taka sytuacja może mieć miejsce przy występowaniu milionów lub miliardów cech.

Techniki regularyzacyjne działają poprzez 
- karanie wielkości współczynników cech, 
- minimalizowanie błędu między przewidywanymi a rzeczywistymi obserwacjami.

## Dlaczego karamy za wielkość współczynników?

Rozważmy następujący przykład celem zrozumienia wpływu złożoności modelu na wielkość współczynników.

W tym celu dopasujmy krzywą regresji do krzywej sinusoidalnej (od 0° do 360°) z dodanym szumem.

In [ ]:
# np.random.seed(123) 

In [ ]:
# #generacja danych
# x = np.array([i*np.pi/180 for i in range(0,360,1)])
# X = pd.DataFrame(x)
# y = np.sin(x)+np.random.normal(0,0.2,len(x))
# plt.plot(x,y,'.',color = 'black')

In [ ]:
# #lambda to taka prosta funkcja, bierze dowolną liczbę argumentów, ale może mieć w sobie tylko jedno wyrażenie
# rss_fun  = lambda y, y_pred: sum((y_pred-y)**2)

In [ ]:
# #funkcja tworzy model regresji wielomianowej bez regularyzacji, dopasowuje model do danych i rysuje wykres dla dopasowanych wartości w modelu
# #zwraca RSS, wyraz wolny i resztę współczynników 
# def linear_regression(X, y, power, models_to_plot):
#     reg = make_pipeline(PolynomialFeatures(power),
#                         StandardScaler(),
#                         LinearRegression())
#     reg.fit(X, y)
#     y_pred = reg.predict(X)
    
#     if power in models_to_plot:
#         plt.subplot(models_to_plot[power])
#         plt.tight_layout()
#         plt.plot(x, y_pred)
#         plt.plot(x, y, '.')
#         plt.title('Plot for power: %d' % power)
    
#     ret = [rss_fun(y, y_pred)]
#     ret.extend([reg.named_steps['linearregression'].intercept_])
#     ret.extend(reg.named_steps['linearregression'].coef_[1:])
#     return ret

In [ ]:
# #rysujemy wykres regresji wielomianowej bez regularyzacji, dla różnych potęg
# col = ['RSS', 'Intercept'] + ['coef_x_%d' % i for i in range(1, 16)]
# ind = ['model_pow_%d' % i for i in range(1, 16)]
# coef_matrix_simple = pd.DataFrame(index=ind, columns=col)

# models_to_plot = {1:231, 3:232, 6:233, 9:234, 12:235, 15:236}

# for i in range(1, 16):
#     coef_matrix_simple.iloc[i-1, 0:i+2] = linear_regression(X, y, power=i, models_to_plot=models_to_plot)

In [ ]:
# pd.options.display.float_format = '{:,.2g}'.format
# coef_matrix_simple

**Podsumowanie**:

- wielkość współczynników regresji rośnie eksponencjalnie wraz ze wzrostem złożoności modelu,
- wielkość współczynnika regresji wpływa na istotność zmiennej odpowiadającej temu współczynnikowi w oszacowaniu wielkość zmiennej odpowiedzi, ale gdy wielkość współczynnik jest zbyt duża, algorytm modeluje skomplikowane relacje w celu oszacowania wyników, co często kończy się zbytnim dopasowaniem do danych,

# Regularyzacja grzbietowa (ang. *rigde regression*)

Metoda najmniejszych kwadratów z regularyzacją $l2$, minimalizuje **funkcję kryterialną**:

$$||y - Xb||^2_2 + \alpha \cdot ||b||^2_2,$$

gdzie dla dowolnego wektora $n$-wymiarowego $a = (a_1, a_2, \ldots, a_n)$ zachodzi: $||a||_2 = \sqrt{\sum_{i=1}^n a_i^2}$.

$\alpha$ - siła regularyzacja, $\alpha > 0$ 

* gdy $\alpha = 0$ -- problem uprasza się do zwykłej regresji
* gdy $\alpha = +\infty$ -- współczynnik są równe zeru

## Zadanie 1
Napisz funkcję, która dla dowolnego zbioru ($X$ i $y$) oraz stopnia wielomianu dopasuje model regresji wielomianowej z regularyzacją Ridge z danym parametrem $\alpha$. Ponadto, funkcja narysuje wykres rozproszenia i dopasowaną funkcję regresji dla $k$ danych wartości parametru $\alpha$ przy ustalonym stopniu wielomianu (parametr 
`models_to_plot`).

Następnie wyznacz ramkę danych `coef_matrix_ridge` dla ustalonego stopnia wielomianu (np. 15), której wiersze dla ustalonej będą zawierały: wartość RSS oraz kolejne wartości współczynników regresji dla różnych parametrów $\alpha$, np. lista `alpha_ridge`.

Sprawdź jak zmieniają się wartości współczynników regresji z regularyzacją grzbietową wraz ze zmianą parametru $\alpha$.

In [ ]:
# #analogiczna funkcja co wcześniej, ale teraz dla Ridge
# from sklearn.linear_model import Ridge

# def ridge_regression(X, y, alpha, power, models_to_plot={}):
#     ridgereg = make_pipeline(PolynomialFeatures(power),
#                         StandardScaler(),
#                         Ridge(alpha = alpha))
#     ridgereg.fit(X, y)
#     y_pred = ridgereg.predict(X)
    
#     if alpha in models_to_plot:
#         plt.subplot(models_to_plot[alpha])
#         plt.tight_layout()
#         plt.plot(x, y_pred)
#         plt.plot(x, y, '.')
#         plt.title('Plot for alpha: %.3g' % alpha)
    
#     ret = [rss_fun(y, y_pred)]
#     ret.extend([ridgereg.named_steps['ridge'].intercept_])
#     ret.extend(ridgereg.named_steps['ridge'].coef_[1:])
#     return ret

In [ ]:
# #rysujemy wykres regresji wielomianowej z regularyzacją ridge,dla różnych alph dla potęgi 15
# alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

# col = ['RSS','Intercept'] + ['coef_x_%d'%i for i in range(1, 16)]
# ind = ['alpha_%.2g' % alpha_ridge[i] for i in range(0, len(alpha_ridge))]
# coef_matrix_ridge = pd.DataFrame(index=ind, columns=col)

# models_to_plot = {1e-15:231, 1e-10:232, 1e-4:233, 1e-3:234, 1e-2:235, 5:236}


# power = 15
# for i in range(10):
#     coef_matrix_ridge.iloc[i,] = ridge_regression(X, y, alpha_ridge[i], power, models_to_plot)

In [ ]:
# pd.options.display.float_format = '{:,.2g}'.format
# coef_matrix_ridge

**Podsumowanie**:

- wielkość RSS (suma kwadratów błędów) rośnie wraz ze wzrostem wartości $\alpha$, wraz z redukcją złożoności modelu,
- $\alpha = 1e-15$ daje istotną redukcję wielkości współczynników regresji,
- wyższe wartości $\alpha$ prowadzą do niedouczenia modelu (gwałtowny wzrost RSS dla $\alpha > 1$.
- wiele współczynników jest bardzo małych, ale nie równych zeru.

# Regularyzacja Lasso (ang. *Lasso regression*)

LASSO - Least Absolute Shrinkage and Selection Operator

Metoda najmniejszych kwadratów z regularyzacją $l1$, minimalizuje **funkcję kryterialną**:

$$||y - Xb||^2_2 + \alpha \cdot ||b||_1,$$

gdzie dla dowolnego wektora $n$-wymiarowego $a = (a_1, a_2, \ldots, a_n)$ zachodzi: $||a||_1 = \sum_{i=1}^n |a_i|$.

## Zadanie 2
Napisz funkcję, która dla dowolnego zbioru ($X$ i $y$) oraz stopnia wielomianu dopasuje model regresji wielomianowej z regularyzacją Lasso z danym parametrem $\alpha$. Ponadto, funkcja narysuje wykres rozproszenia i dopasowaną funkcję regresji dla $k$ danych wartości parametru $\alpha$ przy ustalonym stopniu wielomianu (parametr 
`models_to_plot`).

Następnie wyznacz ramkę danych `coef_matrix_ridge` dla ustalonego stopnia wielomianu (np. 15), której wiersze dla ustalonej będą zawierały: wartość RSS oraz kolejne wartości współczynników regresji dla różnych parametrów $\alpha$, np. lista `alpha_ridge`.

Sprawdź jak zmieniają się wartości współczynników regresji z regularyzacją Lasso wraz ze zmianą parametru $\alpha$.

In [ ]:
# #definiujemy funkcję, która dopasowuje model lasso dla regresji wielomianowej i tworzy wykresy dla pewnych alph, zwraca współczynniki i RSS

# from sklearn.linear_model import Lasso

# def lasso_regression(X, y, alpha, power, models_to_plot={}):
#     lassoreg = make_pipeline(PolynomialFeatures(power),
#                         StandardScaler(),
#                         Lasso(alpha = alpha,max_iter = 1000000))
#     lassoreg.fit(X, y)
#     y_pred = lassoreg.predict(X)
    
#     if alpha in models_to_plot:
#         plt.subplot(models_to_plot[alpha])
#         plt.tight_layout()
#         plt.plot(x, y_pred)
#         plt.plot(x, y, '.')
#         plt.title('Plot for alpha: %.3g' % alpha)
    
#     ret = [rss_fun(y, y_pred)]
#     ret.extend([lassoreg.named_steps['lasso'].intercept_])
#     ret.extend(lassoreg.named_steps['lasso'].coef_[1:])
#     return ret    

In [ ]:
# #rysujemy wykres regresji wielomianowej z regularyzacją lasso, dla różnych alph dla potęgi 15
# alpha_lasso = [1e-15, 1e-10, 1e-8,1e-5, 1e-4, 1e-3,1e-2, 1, 5, 10]

# col = ['RSS','Intercept'] + ['coef_x_%d'%i for i in range(1, 16)]
# ind = ['alpha_%.2g' % alpha_lasso[i] for i in range(0, len(alpha_lasso))]
# coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

# models_to_plot = {1e-15:231, 1e-10:232, 1e-4:233, 1e-3:234, 1e-2:235, 5:236}


# power = 15
# for i in range(10):
#     coef_matrix_lasso.iloc[i,] = lasso_regression(X, y, alpha_lasso[i], power, models_to_plot)

In [ ]:
# pd.options.display.float_format = '{:,.2g}'.format
# coef_matrix_lasso

**Podsumowanie**:

- wielkość RSS (suma kwadratów błędów) rośnie wraz ze wzrostem wartości 𝛼, wraz z redukcją złożoności modelu,
- dla tych samym wartości $\alpha$, wielkość współczynników regresji z regularyzacją Lasso jest mniejsza niż wartości odpowiadających współczynników w regresji z regularyzacją grzbietową,
- dla tych samych wartości $\alpha$ regresji z regularyzacją Lasso ma wyższe RSS w porównaniu do regresji z regularyzacją grzbietową (gorsze dopasowanie modelu),
- wiele współczynników jest zerowa (nawet dla niewielkich wielkości $\alpha$).

## Porównanie regularyzacji grzbietowej z regularyzacją Lasso

### Ridge
- zawiera wszystkie (lub żadne) cechy w modelu, główną zaletą tej regularyzacji jest **ściągniecie współczynników** (ang. **shrinkage coefficient**),
- regresji grzbietowej używa się głowniej do **uniknięcia przeuczenia** modelu, ale z racji, że zawiera wszystkie zmienne z modelu nie jest użyteczny w przypadku wielowymiarowych danych (gdy liczbę predyktorów szacuje się milionach/miliardach -- zbyt duża złożoność obliczeniowa),
- zasadniczo działa dobrze nawet w obecności silnie **skorelowanych** cech -- uwzględnia wszystkie skorelowane zmienne w modelu, ale wielkość współczynników zależy od wielkości korelacji.


### Lasso 
- regularyzacja Lasso poza **ściągniecie współczynników**, dokonuje również selekcji zmiennych
- regularyzacje Lasso często wykorzystuje się do **selekcji zmiennych** w przypadku do liczba cech jest rzędu milionów/miliardów
- wybiera dowolną cechę spośród cech silnie skorelowanych, współczynniki pozostałych cechy skorelowanych z wybraną zmienną redukuje do zera, ale wybrana zmienna zmienia się losowo wraz ze zmianą parametrów modelu -- podejście te działa gorzej niż regularyzacja grzbietowa